<a href="https://colab.research.google.com/github/garfield-gray/DataMining/blob/main/Frequent_Pattern_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
# Recheck the Preprocessing based on the new data info and based on the needs!
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns


#Theoretical

#Imperical

##A

In [ ]:
#Going into the directory
#    "Address of files"
%cd /content/drive/MyDrive/Uni/Data/DataMining/Winter2024/CA#3
!ls

/content/drive/MyDrive/Uni/Data/DataMining/Winter2024/CA#3
 CA#3Report.docx  'Frequent Pattern Mining.ipynb'   Groceries.csv


In [ ]:

spark = SparkSession.builder.appName(
    'Read CSV File into DataFrame').getOrCreate()

Groceries = spark.read.csv('Groceries.csv', sep=',',
                         inferSchema=True, header=True)

In [ ]:
Groceries.show(n=5)

+-------------+----------+----------------+
|Member_number|      Date| itemDescription|
+-------------+----------+----------------+
|         1808|21-07-2015|  tropical fruit|
|         2552|05-01-2015|      whole milk|
|         2300|19-09-2015|       pip fruit|
|         1187|12-12-2015|other vegetables|
|         3037|01-02-2015|      whole milk|
+-------------+----------+----------------+
only showing top 5 rows



In [ ]:
Groceries.count()

38765

In [ ]:
GroceriesItems = Groceries.drop('Date')

In [ ]:
GroceriesItems.show()

+-------------+--------------------+
|Member_number|     itemDescription|
+-------------+--------------------+
|         1808|      tropical fruit|
|         2552|          whole milk|
|         2300|           pip fruit|
|         1187|    other vegetables|
|         3037|          whole milk|
|         4941|          rolls/buns|
|         4501|    other vegetables|
|         3803|          pot plants|
|         2762|          whole milk|
|         4119|      tropical fruit|
|         1340|        citrus fruit|
|         2193|                beef|
|         1997|         frankfurter|
|         4546|             chicken|
|         4736|              butter|
|         1959|fruit/vegetable j...|
|         1974|packaged fruit/ve...|
|         2421|           chocolate|
|         1513|       specialty bar|
|         1905|    other vegetables|
+-------------+--------------------+
only showing top 20 rows



##B, C, D

In [ ]:
MemeberGroceries = GroceriesItems.groupBy("Member_number").agg(F.concat_ws(", ", F.collect_list(GroceriesItems.itemDescription)))
MemeberGroceries.show()

+-------------+--------------------------------------------+
|Member_number|concat_ws(, , collect_list(itemDescription))|
+-------------+--------------------------------------------+
|         1000|                        soda, canned beer...|
|         1001|                        frankfurter, fran...|
|         1002|                        tropical fruit, b...|
|         1003|                        sausage, root veg...|
|         1004|                        other vegetables,...|
|         1005|                        whipped/sour crea...|
|         1006|                        whole milk, frank...|
|         1008|                        hamburger meat, t...|
|         1009|                        herbs, pastry, tr...|
|         1010|                        pip fruit, frankf...|
|         1011|                        whole milk, frank...|
|         1012|                        frankfurter, proc...|
|         1013|                        frozen meals, whi...|
|         1014|         

In [ ]:
GroceriesItems.where(GroceriesItems.Member_number == 1002).show()

+-------------+-------------------+
|Member_number|    itemDescription|
+-------------+-------------------+
|         1002|     tropical fruit|
|         1002|        butter milk|
|         1002|             butter|
|         1002|  frozen vegetables|
|         1002|              sugar|
|         1002|specialty chocolate|
|         1002|         whole milk|
|         1002|   other vegetables|
+-------------+-------------------+



In [ ]:
MemeberGroceries.collect()[2][1]

'tropical fruit, butter milk, butter, frozen vegetables, sugar, specialty chocolate, whole milk, other vegetables'

In [ ]:
MemeberGroceries.where(MemeberGroceries.Member_number == 1002).show()

+-------------+--------------------------------------------+
|Member_number|concat_ws(, , collect_list(itemDescription))|
+-------------+--------------------------------------------+
|         1002|                        tropical fruit, b...|
+-------------+--------------------------------------------+



In [ ]:
type(MemeberGroceries.collect()[2][1])

str

In [ ]:
#changing the str to list

MemeberGroceriesList = MemeberGroceries.select(F.col("Member_number"),
                F.split(F.col("concat_ws(, , collect_list(itemDescription))"),",").alias("itemDescription_Arr"),
                ).drop("concat_ws(, , collect_list(itemDescription))")

MemeberGroceriesList.show()

+-------------+--------------------+
|Member_number| itemDescription_Arr|
+-------------+--------------------+
|         1000|[soda,  canned be...|
|         1001|[frankfurter,  fr...|
|         1002|[tropical fruit, ...|
|         1003|[sausage,  root v...|
|         1004|[other vegetables...|
|         1005|[whipped/sour cre...|
|         1006|[whole milk,  fra...|
|         1008|[hamburger meat, ...|
|         1009|[herbs,  pastry, ...|
|         1010|[pip fruit,  fran...|
|         1011|[whole milk,  fra...|
|         1012|[frankfurter,  pr...|
|         1013|[frozen meals,  w...|
|         1014|[whole milk,  can...|
|         1015|[citrus fruit,  s...|
|         1016|[UHT-milk,  soft ...|
|         1017|[soda,  other veg...|
|         1018|[butter milk,  cu...|
|         1019|[hamburger meat, ...|
|         1020|[canned beer,  ca...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
(MemeberGroceriesList.collect()[9][1])

['pip fruit',
 ' frankfurter',
 ' pip fruit',
 ' bottled water',
 ' candles',
 ' bottled water',
 ' coffee',
 ' specialty bar',
 ' kitchen towels',
 ' rolls/buns',
 ' UHT-milk',
 ' sliced cheese']

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.fpm import FPGrowth


# Define a function to remove duplicates from a list
def remove_duplicates(lst):
    return list(set(lst))

# Define the UDF to apply the function
remove_duplicates_udf = udf(remove_duplicates, ArrayType(StringType()))

# Define a function to process DataFrame
def process_dataframe(df):

    return df.withColumn('itemDescription_Arr', remove_duplicates_udf(col('itemDescription_Arr')))


with SparkSession.builder.appName("RemoveDuplicatesExample").getOrCreate() as spark:

    # Process the DataFrame
    processed_dataframe = process_dataframe(MemeberGroceriesList)


    MGLBiggerCutommers = processed_dataframe.filter((F.size(processed_dataframe.itemDescription_Arr) > 10))
    print("Part B")
    MGLBiggerCutommers.show()
    fp = FPGrowth(itemsCol="itemDescription_Arr", minSupport=0.15, minConfidence=0.4)


    fpm = fp.fit(MGLBiggerCutommers.select('itemDescription_Arr'))
    fpm.setPredictionCol("newPrediction")
    print("Part C")
    fpm.freqItemsets.sort("items").show()
    print("Part D")
    fpm.associationRules.sort("antecedent", "consequent").show(20)


Part B
+-------------+--------------------+
|Member_number| itemDescription_Arr|
+-------------+--------------------+
|         1000|[ semi-finished b...|
|         1004|[ tropical fruit,...|
|         1006|[ bottled water, ...|
|         1008|[ tropical fruit,...|
|         1010|[ bottled water, ...|
|         1011|[ bottled water, ...|
|         1012|[ tropical fruit,...|
|         1013|[ bottled water, ...|
|         1023|[ berries, white ...|
|         1026|[ root vegetables...|
|         1028|[ bottled water, ...|
|         1032|[ newspapers,  wh...|
|         1033|[ chicken,  sausa...|
|         1035|[ newspapers,  wh...|
|         1038|[other vegetables...|
|         1050|[ cat food,  past...|
|         1051|[ tropical fruit,...|
|         1052|[ soda,  soft che...|
|         1061|[ root vegetables...|
|         1062|[ tropical fruit,...|
+-------------+--------------------+
only showing top 20 rows

Part C
+--------------------+----+
|               items|freq|
+---------------